## Данные

Данные в [архиве](https://drive.google.com/file/d/15o7fdxTgndoy6K-e7g8g1M2-bOOwqZPl/view?usp=sharing). В нём два файла:
- `news_train.txt` тренировочное множество
- `news_test.txt` тренировочное множество

С некоторых новостных сайтов были загружены тексты новостей за период  несколько лет, причем каждая новость принаделжит к какой-то рубрике: `science`, `style`, `culture`, `life`, `economics`, `business`, `travel`, `forces`, `media`, `sport`.

В каждой строке файла содержится метка рубрики, заголовок новостной статьи и сам текст статьи, например:

>    **sport**&nbsp;&lt;tab&gt;&nbsp;**Сборная Канады по хоккею разгромила чехов**&nbsp;&lt;tab&gt;&nbsp;**Сборная Канады по хоккею крупно об...**

# Задача

1. Обработать данные, получив для каждого текста набор токенов
Обработать токены с помощью (один вариант из трех):
    - pymorphy2
    - русского [snowball стеммера](https://www.nltk.org/howto/stem.html)
    - [SentencePiece](https://github.com/google/sentencepiece) или [Huggingface Tokenizers](https://github.com/huggingface/tokenizers)
    
    
2. Обучить word embeddings (fastText, word2vec, gloVe) на тренировочных данных. Можно использовать [gensim](https://radimrehurek.com/gensim/models/word2vec.html) . Продемонстрировать семантические ассоциации. 

3. Реализовать алгоритм классификации, посчитать точноть на тестовых данных, подобрать гиперпараметры. Метод векторизации выбрать произвольно - можно использовать $tf-idf$ с понижением размерности (см. scikit-learn), можно использовать обученные на предыдущем шаге векторные представления, можно использовать [предобученные модели](https://rusvectores.org/ru/models/). Имейте ввиду, что простое "усреднение" токенов в тексте скорее всего не даст положительных результатов. Нужно реализовать два алгоритмов из трех:
     - SVM
     - наивный байесовский классификатор
     - логистическая регрессия
    

4.* Реализуйте классификацию с помощью нейросетевых моделей. Например [RuBERT](http://docs.deeppavlov.ai/en/master/features/models/bert.html) или [ELMo](https://rusvectores.org/ru/models/).

In [1]:
!pip install pymorphy2
!pip install -U pymorphy2-dicts-ru

     |████████████████████████████████| 61kB 3.6MB/s 
     |████████████████████████████████| 8.2MB 6.8MB/s 
Requirement already up-to-date: pymorphy2-dicts-ru in /usr/local/lib/python3.6/dist-packages (2.4.417127.4579844)


In [3]:
import pymorphy2
from google.colab import drive
import random
import numpy as np
from gensim.models import Word2Vec
import warnings
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.naive_bayes import GaussianNB
warnings.filterwarnings('ignore')

drive.mount('/content/drive')
dir_train = "/content/drive/My Drive/data/news/news_train.txt"
dir_test = "/content/drive/My Drive/data/news/news_test.txt"

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [4]:
def data_split(d):
  new_d = []
  
  for row in d:
    new_d.append(row.split('\t'))

  return new_d

In [5]:
train_d = list(open(dir_train, 'r', encoding='utf-8'))
test_d = list(open(dir_test, 'r', encoding='utf-8'))

random.shuffle(train_d)
random.shuffle(test_d)

train_d = data_split(train_d)
test_d = data_split(test_d)



In [5]:
tokenized_train_corpus = [train_d[i][2].split(' ') for i in range(len(train_d))]

In [6]:
morph = pymorphy2.MorphAnalyzer(lang='ru')

for i, row in enumerate(tokenized_train_corpus):
  tokenized_train_corpus[i] = [morph.parse(j)[0].normal_form if morph.parse(j)[0].normal_form else j for j in row]

<h1> word embeddings </h1>

In [7]:
model = Word2Vec(window=3)
model.build_vocab(tokenized_train_corpus)
model.train(tokenized_train_corpus, total_examples=model.corpus_count, epochs=15, report_delay=1)

(32373720, 42788730)

In [8]:
model.wv.most_similar(positive=['мяч'], topn=5)

[('гол', 0.8646789789199829),
 ('шайба', 0.8292951583862305),
 ('ворота', 0.8067502975463867),
 ('овечкин', 0.7717961072921753),
 ('дубль', 0.7694848775863647)]

In [9]:
model.wv.most_similar(positive=['жена'], topn=5)

[('супруг', 0.8710170984268188),
 ('муж', 0.8568508625030518),
 ('дочь', 0.8197134733200073),
 ('подруга', 0.8056074380874634),
 ('отец', 0.7920505404472351)]

In [10]:
model.wv.most_similar(positive=['бег'], topn=5)

[('дистанция', 0.8159993886947632),
 ('заплыв', 0.7514245510101318),
 ('эстафета', 0.7142021656036377),
 ('метр', 0.7054740190505981),
 ('медаль', 0.650199830532074)]

<h1> классификации </h1>

In [11]:
tokenized_test_corpus = [test_d[i][2].split(' ') for i in range(len(test_d))]

for i, row in enumerate(tokenized_test_corpus):
  tokenized_test_corpus[i] = [morph.parse(j)[0].normal_form if morph.parse(j)[0].normal_form else j for j in row]

In [6]:
train_labels = [row[0] for row in train_d]
test_labels = [row[0] for row in test_d]

labels = list(set(train_labels))

dict_labels = {key: value for value, key in enumerate(labels)}

train_labels = [dict_labels[label] for label in train_labels]
test_labels = [dict_labels[label] for label in test_labels]

In [29]:
train_data = np.zeros((len(tokenized_train_corpus), 100))
test_data = np.zeros((len(tokenized_test_corpus), 100))

for i, row in enumerate(tokenized_train_corpus):
  sentence = np.zeros((100,))

  for word in row:

    try:
      vec = model.wv[word]
      sentence += vec
    except:
      continue

  train_data[i, :] += sentence


for i, row in enumerate(tokenized_test_corpus):
  sentence = np.zeros((100,))

  for word in row:

    try:
      vec = model.wv[word]
      sentence += vec
    except:
      continue

  test_data[i, :] += sentence

In [35]:
lin_reg = LogisticRegression(multi_class="multinomial").fit(train_data, train_labels)
lin_reg.score(test_data, test_labels)

0.8416666666666667

In [36]:
svm = SVC().fit(train_data, train_labels)
svm.score(test_data, test_labels)

0.8446666666666667

In [39]:
nb = GaussianNB().fit(train_data, train_labels)
nb.score(test_data, test_labels)

0.656

<h1> RuBERT </h1>

In [3]:
!pip install deeppavlov
!pip install pytorch-transformers
!pip install transformers

In [2]:
from deeppavlov.models.preprocessors.torch_bert_preprocessor import TorchBertPreprocessor
from deeppavlov.models.torch_bert.torch_bert_classifier import TorchBertClassifierModel
import torch
from torch.utils.data import TensorDataset, DataLoader, RandomSampler

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package perluniprops to /root/nltk_data...
[nltk_data]   Package perluniprops is already up-to-date!
[nltk_data] Downloading package nonbreaking_prefixes to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package nonbreaking_prefixes is already up-to-date!


In [7]:
test_data_to_rubert = [row[2] for row in test_d]
train_data_to_rubert = [row[2] for row in train_d]

In [10]:
torch_prepro = TorchBertPreprocessor('DeepPavlov/rubert-base-cased', do_lower_case=True, max_seq_length=32)
tokenized_data = [torch_prepro(sentence) for sentence in train_data_to_rubert]

train_inputs = torch.tensor(tokenized_data)
train_labels = torch.tensor(train_labels)

train_data = TensorDataset(train_inputs, train_labels)
train_dataloader = DataLoader( train_data, sampler = RandomSampler(train_data), batch_size = 32)

Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.


KeyboardInterrupt: ignored